# Google Colab MNIST test

Results :

| Material         | Train time/step | Eval time/step |
| ---------------- | --------------- | -------------- |
| Google TPU              | 50 ms           |   69 ms        |
| GTX 1080         | 16 ms           |   14 ms        |
| i7-8700K         | 250 ms          |   77 ms        |
| Google CPU       | 1 s             |   270 ms        | 
| Google GPU (Tesla K80) |   40 ms            |   18 ms           |

In [1]:
import sys
import os
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.datasets import mnist, fashion_mnist

print(f"tf version : {tf.__version__}\n"
      f"keras version : {keras.__version__}\n"
      f"python version : {sys.version}")

tf version : 1.12.0
keras version : 2.1.6-tf
python version : 3.7.1 (default, Oct 22 2018, 10:41:28) 
[GCC 8.2.1 20180831]


## Check for TPU

In [2]:
def get_tpu_address():
    tpu_address = ""
    try:
        device_name = os.environ['COLAB_TPU_ADDR']
        tpu_address = 'grpc://' + device_name
        print('Found TPU at: {}'.format(tpu_address))
    except KeyError:
        print('TPU not found')
    return tpu_address

tpu_address = get_tpu_address()

TPU not found


In [3]:
# Run this cell to disable gpu
os.environ['CUDA_VISIBLE_DEVICES'] = ''

## Prepare MNIST

In [4]:
batch_size = 1024
num_classes = 10
epochs = 5
learning_rate = 0.001

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [6]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [7]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [8]:
def train_input_fn(batch_size=1024):
    dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train))
    dataset = dataset.shuffle(1000).repeat().batch(batch_size, drop_remainder=True)
    return dataset


def test_input_fn(batch_size=1024):
    dataset = tf.data.Dataset.from_tensor_slices((x_test,y_test))
    dataset = dataset.shuffle(1000).repeat().batch(batch_size, drop_remainder=True)
    return dataset



## Prepare model

In [9]:
Input = tf.keras.Input
Conv2D = tf.keras.layers.Conv2D
MaxPooling2D = tf.keras.layers.MaxPooling2D
Flatten = tf.keras.layers.Flatten
Dropout = tf.keras.layers.Dropout
Dense = tf.keras.layers.Dense

inputs = Input(name='input', 
               shape=input_shape, 
               batch_size=batch_size, 
               dtype=tf.float32)

x = Conv2D(32, kernel_size=(3, 3), activation='relu',name = 'Conv_01')(inputs)
x = MaxPooling2D(pool_size=(2, 2),name = 'MaxPool_01')(x)
x = Conv2D(64, (3, 3), activation='relu',name = 'Conv_02')(x)
x = MaxPooling2D(pool_size=(2, 2),name = 'MaxPool_02')(x)
x = Conv2D(64, (3, 3), activation='relu',name = 'Conv_03')(x)
x = Flatten(name = 'Flatten_01')(x)
x = Dense(64, activation='relu',name = 'Dense_01')(x)
x = Dropout(0.5,name = 'Dropout_02')(x)

output = Dense(num_classes, activation='softmax',name = 'Dense_02')(x)

model = tf.keras.Model(inputs=[inputs], outputs=[output])

In [10]:
# use a tf optimizer rather than a Keras one for now
opt = tf.train.AdamOptimizer(learning_rate)

model.compile(
    optimizer=opt,
    loss='categorical_crossentropy',
    metrics=['acc'])


## Train and evaluate on TPU

In [ ]:
tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)))

In [ ]:
tpu_model.summary()

In [ ]:
tpu_model.fit(
  train_input_fn,
  steps_per_epoch = 60,
  epochs=10,
)

In [ ]:
tpu_model.evaluate(test_input_fn, steps = 100)

## Train and evaluate on GPU/CPU

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (1024, 28, 28, 1)         0         
_________________________________________________________________
Conv_01 (Conv2D)             (1024, 26, 26, 32)        320       
_________________________________________________________________
MaxPool_01 (MaxPooling2D)    (1024, 13, 13, 32)        0         
_________________________________________________________________
Conv_02 (Conv2D)             (1024, 11, 11, 64)        18496     
_________________________________________________________________
MaxPool_02 (MaxPooling2D)    (1024, 5, 5, 64)          0         
_________________________________________________________________
Conv_03 (Conv2D)             (1024, 3, 3, 64)          36928     
_________________________________________________________________
Flatten_01 (Flatten)         (1024, 576)               0         
__________

In [12]:
model.fit(
  train_input_fn(),
  steps_per_epoch = 60,
  epochs=10,
)

Epoch 1/10
60/60 [==============================] - 16s 259ms/step - loss: 1.1162 - acc: 0.6367
Epoch 2/10
60/60 [==============================] - 16s 268ms/step - loss: 0.3684 - acc: 0.8883
Epoch 3/10
60/60 [==============================] - 15s 248ms/step - loss: 0.2397 - acc: 0.9285
Epoch 4/10
60/60 [==============================] - 15s 248ms/step - loss: 0.1890 - acc: 0.9454
Epoch 5/10
60/60 [==============================] - 15s 252ms/step - loss: 0.1558 - acc: 0.9543
Epoch 6/10
60/60 [==============================] - 15s 246ms/step - loss: 0.1323 - acc: 0.9618
Epoch 7/10
60/60 [==============================] - 15s 246ms/step - loss: 0.1185 - acc: 0.9655
Epoch 8/10
60/60 [==============================] - 15s 253ms/step - loss: 0.1057 - acc: 0.9685
Epoch 9/10
60/60 [==============================] - 15s 246ms/step - loss: 0.0972 - acc: 0.9721
Epoch 10/10
60/60 [==============================] - 15s 253ms/step - loss: 0.0864 - acc: 0.9758


In [13]:
model.evaluate(test_input_fn(), steps = 100)

100/100 [==============================] - 8s 77ms/step


[0.036014498695731166, 0.987919921875]